In [1]:
import os
import glob
import copy
import pandas as pd
import pylab as pl
import seaborn as sns

import psignifit as ps
import _pickle as pkl


In [2]:
import analyze2p.plotting as pplot

/n/coxfs01/2p-pipeline/envs/rat2p/lib/python3.7/site-packages/outdated/utils.py:18: OutdatedPackageWarning: The package pingouin is out of date. Your version is 0.3.10, the latest is 0.4.0.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  **kwargs


In [4]:
src_dir = os.path.join('/n/coxfs01/behavior-data/threeport')
fit_dir = os.path.join(src_dir, 'processed/morphs/fits')

In [6]:
os.listdir(fit_dir)

['fits_gauss_AG2.pkl',
 'fits_gauss_AG3.pkl',
 'fits_gauss_AG4.pkl',
 'fits_gauss_AG6.pkl',
 'fits_gauss_AG8.pkl',
 'fits_gauss_AG9.pkl',
 'fits_gauss_AG10.pkl',
 'fits_gauss_AG11.pkl',
 'fits_gauss_AN3.pkl',
 'fits_gauss_AN4.pkl',
 'fits_gauss_AN5.pkl',
 'fits_gauss_AN6.pkl',
 'fits_gauss_AN7.pkl',
 'fits_m22-gauss_AG10.pkl',
 'fits_m22-gauss_AG11.pkl',
 'fits_m22-gauss_AG2.pkl',
 'fits_m22-gauss_AG3.pkl',
 'fits_m22-gauss_AG4.pkl',
 'fits_m22-gauss_AG6.pkl',
 'fits_m22-gauss_AG8.pkl',
 'fits_m22-gauss_AG9.pkl',
 'fits_m22-gauss_AN3.pkl',
 'fits_m22-gauss_AN4.pkl',
 'fits_m22-gauss_AN5.pkl',
 'fits_m22-gauss_AN6.pkl',
 'fits_m22-gauss_AN7.pkl',
 'fits_m22-gauss_AG10.pkl.npy']

In [10]:
param_names = ['threshold', 'width', 'lambda', 'gamma', 'eta']


In [8]:
prefix = 'fits_m22-gauss'
aggr_fpath = glob.glob(os.path.join(fit_dir, 'aggr_%s*.pkl' % prefix))[0]

IndexError: list index out of range

In [22]:
animalid = os.path.splitext(os.path.split(fn)[-1])[0].split(prefix)[-1][1:]



In [24]:
fpaths = glob.glob(os.path.join(fit_dir, '%s*.pkl' % prefix))
print(len(fpaths))

d_=[]
for fn in fpaths:
    animalid = os.path.splitext(os.path.split(fn)[-1])[0].split(prefix)[-1][1:]
    with open(fn, 'rb') as f:
        res = pkl.load(f)    
    df_ = pd.DataFrame(res['Fit'], index=param_names, columns=[animalid]).T
    try:
        thr = ps.getThreshold(res, 0.5, 0)[0] # Value at which function reaches at_pc correct
        slp = ps.getSlope(res, ps.getThreshold(res, 0.5, 1)[0]) # Slope at given stimulus level
    except Exception as e:
        thr=None
        slp=None
    [thr_0,CI0] = ps.getThreshold(res,0.25,1)
    [thr_1,CI1] = ps.getThreshold(res,0.75,1)
    jnd = thr_1-thr_0
    df_['jnd'] = jnd
    df_['slope'] = slp
    df_['thr_scaled'] = thr
    df_['animalid'] = animalid
    d_.append(df_)

13


In [25]:
FITS = pd.concat(d_, axis=0)
